# Select training data from large CML dataset
This code can only be run if you have the large CML dataset. 

In [1]:
import numpy as np
import xarray as xr
from sklearn.metrics import matthews_corrcoef

## For rain gauge reference

In [2]:
# Read CML data with data from nearest rain gauge
ds_cmls = xr.open_dataset('/home/erlend/delete/cml_data_germany.nc')

In [3]:
# Rename variables from German dataset to make it run on this code
ds_cmls = ds_cmls.rename({'channel_id': 'sublink_id'})

In [4]:
# Select training data using RSD and the MCC metric.
roll_std_dev = ds_cmls.trsl.rolling(time=60, center=True).std() 

threshold = 1.12*roll_std_dev.quantile(0.8,dim='time')

ds_cmls["wet_graf"] = roll_std_dev>threshold

keep = []
mccs = []
rg_used = [] # ensure that we use unique rain gauges
for cml_id in ds_cmls.cml_id:
    mcc_1 = matthews_corrcoef(ds_cmls.sel(cml_id = cml_id).gauge_wet, ds_cmls.sel(cml_id = cml_id).wet_graf.isel(sublink_id = 0 ))
    mcc_2 = matthews_corrcoef(ds_cmls.sel(cml_id = cml_id).gauge_wet, ds_cmls.sel(cml_id = cml_id).wet_graf.isel(sublink_id = 1 ))
    ch_1 =  mcc_1 > 0.3 
    ch_2 =  mcc_2 > 0.3

    # Use CML if both channels have a good mcc and the rain gauge has not been used before
    if ch_1 and ch_2 and (ds_cmls.sel(cml_id = cml_id).gauge_id.values not in rg_used):# and (rainammount < 0.20) and (rainammount > 0.01):
        rg_used.append(ds_cmls.sel(cml_id = cml_id).gauge_id.values)
        keep.append(cml_id.values) # record name of cml
        mccs.append(mcc_1) # record mcc from channel 1

# get the 25 largest mccs
indices = np.argpartition(mccs, -26)[-26:]
keep = np.array(keep)[indices]

# Select training data
ds_cmls = ds_cmls.sel(cml_id = keep)

/home/erlend/miniforge3/envs/pycomlink-dev/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1634: RuntimeWarning: All-NaN slice encountered
  return fnb._ureduce(a,


In [5]:
# Rename CMLs
ds_cmls['cml_id'] = np.arange(ds_cmls.cml_id.size).astype(str)

# Remove coordinates
ds_cmls = ds_cmls.reset_coords(drop=True)

In [6]:
# Save CML data 
ds_cmls.to_netcdf("train_rg.nc")

## For radar reference

In [7]:
# Read CML data with data from weather radar along CML
ds_cmls = xr.open_dataset('/home/erlend/delete/cml_data_germany.nc')

In [8]:
# Rename variables from German dataset to make it run on this code
ds_cmls = ds_cmls.rename({'channel_id': 'sublink_id'})

In [9]:
# Select training data using RSD and the MCC metric.
roll_std_dev = ds_cmls.trsl.rolling(time=60, center=True).std() 

threshold = 1.12*roll_std_dev.quantile(0.8,dim='time')

ds_cmls["wet_graf"] = roll_std_dev>threshold

keep = []
mccs = []
rg_used = [] # ensure that we use unique rain gauges
for cml_id in ds_cmls.cml_id:
    mcc_1 = matthews_corrcoef(ds_cmls.sel(cml_id = cml_id).radar_wet, ds_cmls.sel(cml_id = cml_id).wet_graf.isel(sublink_id = 0 ))
    mcc_2 = matthews_corrcoef(ds_cmls.sel(cml_id = cml_id).radar_wet, ds_cmls.sel(cml_id = cml_id).wet_graf.isel(sublink_id = 1 ))
    ch_1 =  mcc_1 > 0.3 
    ch_2 =  mcc_2 > 0.3

    # Use CML if both channels have a good mcc and the rain gauge has not been used before
    if ch_1 and ch_2 and (ds_cmls.sel(cml_id = cml_id).gauge_id.values not in rg_used):# and (rainammount < 0.20) and (rainammount > 0.01):
        rg_used.append(ds_cmls.sel(cml_id = cml_id).gauge_id.values)
        keep.append(cml_id.values) # record name of cml
        mccs.append(mcc_1) # record mcc from channel 1

# get the 25 largest mccs
indices = np.argpartition(mccs, -26)[-26:]
keep = np.array(keep)[indices]

# Select training data
ds_cmls = ds_cmls.sel(cml_id = keep)

/home/erlend/miniforge3/envs/pycomlink-dev/lib/python3.12/site-packages/numpy/lib/_nanfunctions_impl.py:1634: RuntimeWarning: All-NaN slice encountered
  return fnb._ureduce(a,


In [10]:
# Rename CMLs
ds_cmls['cml_id'] = np.arange(ds_cmls.cml_id.size).astype(str)

# Remove coordinates
ds_cmls = ds_cmls.reset_coords(drop=True)

In [11]:
# Save CML data 
ds_cmls.to_netcdf("train_rad.nc")